In [1]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import os
import joblib

from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from catboost import CatBoostRegressor

2025-01-28 20:35:43.431492: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 20:35:43.438626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738067743.446084   34443 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738067743.448365   34443 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 20:35:43.457671: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
path = os.getcwd() + "/data/data.csv"
df = pd.read_csv(path)
df[['gu', 'ro']] = df['Address'].str.split(' ', expand=True).iloc[:, :2]
df['Supply_type'] = df['Supply_type'].str.replace(r'\D', '', regex=True)
qty = (3 - df['Cutline_rate']) * 10 + df['Cutline_score']

df.drop(
    columns=[
        'Name','Address', 'Latitude', 'Longitude', 'Infra_score',
        'Gender','Shared','Quarter','Counts_supermarket','Counts_laundry',
        'Counts_pharmacy','Cutline_rate','Cutline_score'
    ],
    inplace=True
)

df = pd.get_dummies(data=df)
df['Qty'] = qty

In [3]:
X = df.drop(columns='Qty')
y = df['Qty']

oversampler = RandomOverSampler(sampling_strategy={2: 500, 3: 500})
X, y = oversampler.fit_resample(X, y)

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2
)

model = CatBoostRegressor(
    iterations=1000,
    depth=6,
    learning_rate=0.1,
    loss_function='RMSE',
    verbose=100,
    task_type='CPU'
)

model.fit(
    X, y,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=10,
    use_best_model=True,
    plot=True
)

y_pred = model.predict(X_valid)

# print('Rate : ')
# print(classification_report(y_valid,y_pred))
# print('Rate CM:')
# print(confusion_matrix(y_valid,y_pred))
print('Score MAE:',mean_absolute_error(y_valid,y_pred))
print('Score R2:',r2_score(y_valid,y_pred))

joblib.dump(model, os.getcwd() + '/main.pkl')


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 9.7540688	test: 9.5357939	best: 9.5357939 (0)	total: 48.9ms	remaining: 48.9s
100:	learn: 2.2789929	test: 2.1316792	best: 2.1316792 (100)	total: 158ms	remaining: 1.41s
200:	learn: 1.8902906	test: 1.7764108	best: 1.7764108 (200)	total: 268ms	remaining: 1.06s
300:	learn: 1.6555219	test: 1.5417636	best: 1.5417636 (300)	total: 371ms	remaining: 862ms
400:	learn: 1.4836872	test: 1.3754557	best: 1.3754557 (400)	total: 470ms	remaining: 701ms
500:	learn: 1.3462702	test: 1.2498104	best: 1.2498104 (500)	total: 586ms	remaining: 584ms
600:	learn: 1.2286325	test: 1.1394143	best: 1.1394143 (600)	total: 684ms	remaining: 454ms
700:	learn: 1.1279744	test: 1.0435107	best: 1.0435107 (700)	total: 778ms	remaining: 332ms
800:	learn: 1.0373888	test: 0.9484078	best: 0.9484078 (800)	total: 879ms	remaining: 218ms
900:	learn: 0.9616919	test: 0.8789228	best: 0.8789228 (900)	total: 990ms	remaining: 109ms
999:	learn: 0.8947782	test: 0.8276868	best: 0.8276868 (999)	total: 1.09s	remaining: 0us

bestTest = 0.8